# You Only Look Once

paper的翻译笔记

http://arxiv.org/abs/1506.02640

2. Unified Detection

We unify the separate components of object detection into a single neural network. Our network uses features from the entire image to predict each bounding box. It also predicts all bounding boxes across all classes for an image simultaneously. This means our network reasons globally about the full image and all the objects in the image. The YOLO design enables end-to-end training and realtime speeds while maintaining high average precision.

我们把物体检测中用到的独立的几个组件统一成了一个单独的神经网络。我们的网络使用来自整个图片的特征去预测每个边界框。同时它还预测每个边界框和每种物体分类。这意味着我们的网络对整个图片和图片中所有的物体是从全局上理解的。YOLO的设计在保持了高平均准确率的同时，也实现了端到端的训练和实时的速度。

Our system divides the input image into an S × S grid. If the center of an object falls into a grid cell, that grid cell is responsible for detecting that object.

我们的系统把输入图片划分成S x S的网格。如果一个物体的中心落在了某一个网格中，这个网格就负责来检测这个物体。

Each grid cell predicts B bounding boxes and confidence scores for those boxes. These confidence scores reflect how
confident the model is that the box contains an object and also how accurate it thinks the box is that it predicts. Formally we define confidence as Pr(Object) ∗ IOU. If no pred object exists in that cell, the confidence scores should be zero. Otherwise we want the confidence score to equal the intersection over union (IOU) between the predicted box and the ground truth.

每一个网格会预测出 B 个边界框和边界框对应的信心值。这个值反应了这个边界框中包含了一个物体，并且这个边界框有多准确的信心。形式上定义这个信心为 Pr(Object) * IOU。如果预测认为框中没有物体，那么这个值就是0。否则我们希望这个值等于预测框和真实框的IOU。

- Pr(Object) 框中有一个物体
- IOU：intersection over union，代表了边界框的准确度。真实的边界框和预测的边界框，这两个框的交集/这两个的并集。

Each bounding box consists of 5 predictions: x, y, w, h, and confidence. The (x, y) coordinates represent the center of the box relative to the bounds of the grid cell. The width and height are predicted relative to the whole image. Finally the confidence prediction represents the IOU between the predicted box and any ground truth box.

每个边界框包含5个预测值：x，y，w，h和信心。坐标（x，y）表示边界框的中心点（相对于当前的网格）。宽和高是相对于整个图片来说的。最后信心表示预测框和真实框的IOU。

Each grid cell also predicts C conditional class probabilities, Pr(Class_i |Object). These probabilities are conditioned on the grid cell containing an object. We only predict one set of class probabilities per grid cell, regardless of the number of boxes B.

每个网格还会预测C个条件概率，Pr(Class_i | Object)。是在网格包含一个物体的条件下某个分类的概率。我们对每个网格只预测了集合中每个分类的概率，而不管这个网格有B个边界框。

At test time we multiply the conditional class probabilities and the individual box confidence predictions,
Pr(Class_i|Object) ∗ Pr(Object) ∗ IOU = Pr(Classi) ∗ IOU which gives us class-specific confidence scores for each box. These scores encode both the probability of that class appearing in the box and how well the predicted box fits the object.

在测试的时候，我们把分类的条件概率和每个编辑框的信心相乘，对每个框我们就知道它包含的分类的信心。这个数值既代表了某个分类物体出现在框中的概率，也代表了框是否很好地“框住”了这个物体。

Figure 2: The Model. Our system models detection as a regression problem. It divides the image into an S × S grid and for each grid cell predicts B bounding boxes, confidence for those boxes, and C class probabilities. These predictions are encoded as an S × S × (B ∗ 5 + C) tensor.

For evaluating YOLO on PASCAL VOC, we use S = 7, B = 2. PASCAL VOC has 20 labelled classes so C = 20. Our final prediction is a 7 × 7 × 30 tensor.

在PASCAL VOC上来评估YOLO时，我们使用了 S = 7，B = 2. PASCAL VOC包含20个分类，所以 C = 20. 我们最终的预测值就是一个 7 x 7 x 30的张量。

2.1. Network Design

We implement this model as a convolutional neural network and evaluate it on the PASCAL VOC detection dataset [9]. The initial convolutional layers of the network extract features from the image while the fully connected layers predict the output probabilities and coordinates.

我们用一个卷积网络来实现这个模型并用PASCAL VOC检测数据库来评估。开始的卷积层用来从图片提取特征，后面的全连接层用来预测概率和坐标。

Our network architecture is inspired by the GoogLeNet model for image classification [34]. Our network has 24 convolutional layers followed by 2 fully connected layers. Instead of the inception modules used by GoogLeNet, we simply use 1 × 1 reduction layers followed by 3 × 3 convolutional layers, similar to Lin et al [22]. The full network is shown in Figure 3.

这种网络结构的设计受到了GoogLeNet图片分类模型的启发。我们的网络有24个卷积层，然后两个全连接层。我们使用1x1的reduction layer后跟3x3的卷积层，而不是GoogLeNet中的inception模块。图片3显示了网络的整体结构。

We also train a fast version of YOLO designed to push the boundaries of fast object detection. Fast YOLO uses a neural network with fewer convolutional layers (9 instead of 24) and fewer filters in those layers. Other than the size of the network, all training and testing parameters are the same between YOLO and Fast YOLO.

我们还设计了另一个快速版本的YOLO网络来做物体的边界检测。Fast YOLO使用一个小一点的神经网络，它有9个而不是24个卷积层，并且每个卷积层的卷积核少一些。除了网络的大小以外，训练和测试中的别的参数都和YOLO一样。

Figure 3: The Architecture. Our detection network has 24 convolutional layers followed by 2 fully connected layers. Alternating 1 × 1 convolutional layers reduce the features space from preceding layers. We pretrain the convolutional layers on the ImageNet classification task at half the resolution (224 × 224 input image) and then double the resolution for detection.

检测网络有24个conv层和2个fc层。交替地使用1x1的卷积层来减低前一层的特征空间。我们使用一半大小的ImageNet分类任务来预训练卷积层（224 x 224的图片输入）然后增大分辨率来做检测。